In [2]:
import xarray
import fsspec
import numpy as np
#import s3fs
import proplot
import os

from matplotlib import pyplot as plt
import pandas

In [3]:
file_path = '/oa-decadal-climate/work/observations/CARSv2_ancillary/MNF_TAS'
file_name = 'MNF_ctd_138.6E_47.9S_155.4E_37.5S.csv'

header = pandas.read_csv(os.path.join(file_path,file_name), index_col=0, nrows=0).columns.tolist()

MNF_CTD_dataframe = pandas.read_csv(os.path.join(file_path,file_name),header=0,usecols=["SURVEY_NAME","STATION",
                                                                                        "START_TIME","END_TIME",
                                                                                        "START_LAT","START_LON",
                                                                                        "END_LAT","END_LON",
                                                                                        "TEMPERATURE","SALINITY","TEMPERATURE_QC","SALINITY_QC","PRESSURE"])

/tube1/cha674/Anaconda_Install/miniconda/envs/py3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
#Loop through each "survey" (ie. cruise) and then each "station" (ie. each individual ctd cast)
#Upack all  the lat, lon and time values into a single long list
#upack all data variables (ie. a temperature cast as a function of pressure) into a list of arrays. 
#Each element in the list is a single cast

survey_identifiers = MNF_CTD_dataframe['SURVEY_NAME'].unique()

latitude  = []
longitude = []
time      = []

pressure  = []
temperature = []
temperature_QC = []

salinity   = []
salinity_QC   = []




for i_survey in survey_identifiers:
    
    dataframe_for_survey = MNF_CTD_dataframe.loc[MNF_CTD_dataframe['SURVEY_NAME']==i_survey]
    stations_for_survey  = dataframe_for_survey['STATION'].unique()
    for i_station in stations_for_survey:
        
        dataframe_for_station = dataframe_for_survey.loc[dataframe_for_survey['STATION']==i_station]
        
        
        
        latitude.append(dataframe_for_station['START_LAT'].unique())
        longitude.append(dataframe_for_station['START_LON'].unique())
        #CTD_depth.append(dataframe_for_station['BOTTOM_DEPTH'].unique())
        
        start_time = pandas.to_datetime(dataframe_for_station['START_TIME'])
        end_time   = pandas.to_datetime(dataframe_for_station['END_TIME'])
        middle_time = (end_time - start_time)/2 + start_time
                                        
        time.append(middle_time.unique())
        
        pressure.append(dataframe_for_station['PRESSURE'].values)
        temperature.append(dataframe_for_station['TEMPERATURE'].values)
        salinity.append(dataframe_for_station['SALINITY'].values)

        temperature_QC.append(dataframe_for_station['TEMPERATURE_QC'].values)
        salinity_QC.append(dataframe_for_station['SALINITY_QC'].values)

        
        
latitude=np.asarray(latitude).squeeze()
longitude=np.asarray(longitude).squeeze()
time = pandas.to_datetime(np.asarray(time).squeeze())

In [5]:
latitude

array([-38.9655 , -44.18917, -47.30017, ..., -43.62772, -46.91397,
       -46.73156])